In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("../data/misgrosi_job.csv")
df

,Position,Company,Location,Job Type
0,"Wir suchen ein zusätzliches Grosi, das uns vor...",Daniela1984,3114 Wichtra,Kinderbetreuung allgemein
1,Suchen liebevolles Grosi für langfristige Betr...,Hendrik,3037 Herrenschwanden,Tages-Grosi
2,Liebe Betreuung gesucht in Rorbas\nFamiliebuehrer,Familiebuehrer,8427 Rorbas,Grosi & Opi
3,Grosi gesucht im Entlebuch\nMamimitHerz,MamimitHerz,6162 Entlebu,Grosi als Nanny/Haushaltshilfe
4,Grosi gesucht für Kinderbetreuung in La Neuvev...,Sina 88,2520 La Neuvevill,Babysitting
...,...,...,...,...
1157,Nachtbetreuung 7 monate altes Baby\nLea G.,Lea G.,8037,Kinderbetreuung allgemein
1158,Familie mit einen Jungen sucht eine Hausaufgab...,Jolanda P.,Luzern,Kinderbetreuung allgemein
1159,Berufstätige Mutter sucht stundenweise Betreuu...,Beatrice S.,Luzern,Kinderbetreuung allgemein
1160,Tages-Grosi für unsere 2 Kinder gesucht.\nLuca...,Lucas2 L.,6500 Bellinzona,Grosi als Nanny/Haushaltshilfe
